In [5]:
from ACNet_Real import ACNet
from ACNet_Real import ReplayMemory
import FireFightingEnv as FFEnv
import torch
import torch.optim as optim
from collections import namedtuple, deque
from itertools import count
import math
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

BATCH_SIZE = 128
GAMMA = 0.99
TAU = 0.005
LR = 1e-6
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 5000
entropy_weight = 0.05
value_loss_weight = 0.5

AGT_COUNT = 3
GRID_SIZE = 7


writer = SummaryWriter('./logsAC/run10')

env = FFEnv.FFEnv(AGT_COUNT)
# Get number of actions from gym action space
n_actions = 10
# Get the number of state observations
state = env.reset()
# n_observations = state
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

AC_net = ACNet(n_actions, GRID_SIZE).to(device)

use_trained_policy = 0
if(use_trained_policy):
    checkpoint = torch.load('./results/checkpoint/Run5.pth')
    AC_net.load_state_dict(checkpoint)
    
target_net = ACNet(n_actions, GRID_SIZE).to(device)
target_net.load_state_dict(AC_net.state_dict())

optimizer = optim.AdamW(AC_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)



cuda


In [6]:
steps_done = 0

def select_action(input, water_res):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    
    if sample > eps_threshold:
        with torch.no_grad():
            action_probs, _ = AC_net(input.unsqueeze(0), water_res)
            return torch.multinomial(action_probs, num_samples=1)

    else:
        return torch.tensor([[random.randint(0, 9)]], device=device, dtype=torch.long)



In [7]:
Transition = namedtuple('Transition',
                        ('state_map', 'state_water', 'action', 'next_state_map', 'next_state_water', 'reward'))


def optimize_model(t):
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    transitions_on_device = [Transition(*(tensor.to(device) for tensor in t)) for t in transitions]
    batch = Transition(*zip(*transitions_on_device))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state_map)), device=device, dtype=torch.bool)
    non_final_next_states_map = torch.stack([s for s in batch.next_state_map
                                                if s is not None], dim=0)
    non_final_next_states_water = torch.cat([s for s in batch.next_state_water
                                                if s is not None])
    state_map_batch = torch.stack(batch.state_map, dim=0)
    state_water_batch = torch.cat(batch.state_water)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    
    action_probs, values = AC_net(state_map_batch, state_water_batch)
    log_probs = torch.log(action_probs + 1e-10)
    selected_log_probs = torch.gather(log_probs, 1, action_batch.view(-1, 1))
    
    entropy = -torch.sum(action_probs * log_probs, dim=1).mean()
    
    advantage = reward_batch - values
    advantage = (advantage - advantage.mean()) / (advantage.std() + 1e-8)

    
    loss = -torch.sum(selected_log_probs * advantage) - entropy_weight * entropy - value_loss_weight * F.smooth_l1_loss(values, reward_batch)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(AC_net.parameters(), 1)
    optimizer.step()

    writer.add_scalar('Loss/Train', loss, t)
    
    return # state_action_values.detach().cpu().numpy().tolist()
    


In [8]:
if torch.cuda.is_available():
    num_episodes = 6000
else:
    num_episodes = 50

DEBUG = False

episode_durations = []
episode_data_list = []

curr_log_step = 0
curr_save_step = 0

for i_episode in range(num_episodes):
    # Initialize the environment and get it's state
    states= env.reset()
    # states = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    # print(states)
    for t in count():
        actions = []
        action_is = []
        water_res_states = []
        input_states = []
        for state in states:
            # state = torch.tensor(state, dtype=torch.float32, device=device)
            input = state[0]
            water_res = state[1]
            water_res = torch.tensor([[water_res]], device=device).float()
            input = torch.from_numpy(input).float().to(device)
            #input = torch.tensor([[input]], device=device).float()
            action_i = select_action(input, water_res)
            action_int = action_i.item()
            action = [action_int % 5, int(action_int / 5)]
            actions.append(action) # For env update
            
            # For network update
            water_res_states.append(water_res)
            input_states.append(input)
            action_is.append(action_i)
            
        observations, rewards, terminated = env.step(actions)
        
        curr_log_step += 1
        writer.add_scalar('Reward/Train', sum(rewards), curr_log_step)
        
        if(DEBUG):
            print(actions)
            print(rewards)
            
        rewards_tensors = []
        for i, reward in enumerate(rewards):
            # print(type(torch.tensor([reward], device=device)))
            rewards_tensors.append(torch.tensor([reward], device=device))
        
        done = terminated

        if terminated:
            next_states = None
        else:
            next_states = observations
            next_input_states = []
            next_water_states = []
            for state in next_states:
                # state = torch.tensor(state, dtype=torch.float32, device=device)
                input = state[0]
                water_res = state[1]
                water_res = torch.tensor([[water_res]]).float()
                input = torch.from_numpy(input).float().to(device)
                
                # For network update
                next_water_states.append(water_res)
                next_input_states.append(input)
                # next_states = torch.tensor(observations, dtype=torch.float32, device=device)
            # Store the transition in memory
            for i in range(AGT_COUNT):
                memory.push(input_states[i], water_res_states[i], action_is[i], next_input_states[i], next_water_states[i], rewards_tensors[i])

            # Move to the next state
            states = next_states

            # Perform one step of the optimization (on the policy network)
            # state_action_values = 
            optimize_model(curr_log_step)
            
            # Soft update of the target network's weights
            # θ′ ← τ θ + (1 −τ )θ′
            target_net_state_dict = target_net.state_dict()
            policy_net_state_dict = AC_net.state_dict()
            for key in policy_net_state_dict:
                target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
            target_net.load_state_dict(target_net_state_dict)

            # episode_data = {
            # 'episode': i_episode,
            # 'state_action_values': state_action_values,
            # #'policy': policy_net.state_dict(),
            # #'optimizer': optimizer.state_dict(),
            # 'reward': sum(rewards).item(),
            # # Add any other information you want to store
            # }
            # episode_data_list.append(episode_data)

            if DEBUG:
                env.mapEnv.plotAll()
                env.mapEnv.plotAgent()
                env.mapEnv.plotFireMap()
                #time.sleep(10) # For Debugging

        
        if done:
            curr_save_step += 1
            if curr_save_step >= 100:
                curr_save_step = 0
                
                torch.save(AC_net.state_dict(), './results/checkpoint/ckpt_AC_cpy_{}.pth'.format(i_episode+1))
                # Convert episode_data_list to a DataFrame
                # state_action_values_list = pd.DataFrame(episode_data_list)
                # Specify the file name
                # csv_file_path = './results/checkpoint/state_action_values_list{}.csv'.format(i_episode+1)
                # Writing to CSV file
                # state_action_values_list.to_csv(csv_file_path, index=False)

        if done:
            # episode_durations.append(t + 1)
            break

/home/luyi/anaconda3/envs/me5418-sb3-test1/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
